In [ ]:
import requests
from bs4 import BeautifulSoup, NavigableString, Tag
import lxml
import json

import unicodedata

# Japanese Text Initiative

Texts of classical Japanese literature are [available](http://jti.lib.virginia.edu/japanese/) at the Japanese Text Initiative sponsored by the University of Virginia.

In [ ]:
base_url = 'http://jti.lib.virginia.edu/japanese/genji/'

In [ ]:
toc_url = 'http://jti.lib.virginia.edu/japanese/genji/tocmodern.html'
s = requests.Session()
r = s.get(toc_url)
GM_toc = r.text

In [ ]:
# Links chapters start at 1 (instead of python 0 based counting).
no_chapters = 54
chapters = [n for n in range(1, no_chapters+1)]
top_level_links = ['Mur' + str(chapter).zfill(2) + 'GM.html' for chapter in chapters]

In [ ]:
top_level_links[:1]

In [ ]:
genji_data = {}
for chapter in chapters:
    chapter_url = base_url + top_level_links[chapter - 1] # Account for 1-based chapter numbering.
    r = s.get(chapter_url)
    
    # HTML(r.text)
    text = r.text
    text = unicodedata.normalize('NFKC', text) # Hex encoded whitespace \x3000
    
    soup = BeautifulSoup(text, 'html.parser')
    
    headings = soup.find_all('h2')
    tags = headings[0].find_all('a')
    chapter_name = tags[0]['name'].strip()
    print(chapter_name, end=' ')
    anchors = []
    for ol in soup.find_all('ol'):
        for anchor in ol.find_all('a'):
            if 'href' in anchor.attrs:
                anchors.append(anchor['href'].replace('#', ''))
                link_url = chapter_url + '#' + anchor['href']
#                 print(anchor.text, link_url)
#         print('='*40)
    paragraphs = soup.find_all('p')
    sections = {}
    for paragraph in paragraphs[1:]:
#         print(len(paragraph.find_all('a')))
        for anchor in paragraph.find_all('a'):
#             print(anchor.attrs)
            if anchor['name'] in anchors:
#                 print(anchor['name'])
                blocks = []
                for tag in anchor:
                    if isinstance(tag, NavigableString):
                        block = str(tag)
                        if block != '\n':
                            blocks.append(block)
#                         print(block)
#                         print('='*40)
                sections[anchor['name']] = blocks
    genji_data[chapter_name] = sections

In [ ]:
print(len(genji_data.keys()))

In [ ]:
with open('../data/raw/genji_data.json', 'w') as fp:
    json.dump(genji_data, fp)